<a href="https://colab.research.google.com/github/steinruck/WANG-690-FALL-2020/blob/master/Final_Project/Deliverable2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

In [48]:
df = pd.read_csv("auth.txt", names=["Date", "IP", "Service", "Description"], delimiter='|')
df

,Date,IP,Service,Description
0,Nov 30 06:39:00,ip-172-31-27-153,CRON[21882],pam_unix(cron:session): session closed for use...
1,Nov 30 06:47:01,ip-172-31-27-153,CRON[22087],pam_unix(cron:session): session opened for use...
2,Nov 30 06:47:03,ip-172-31-27-153,CRON[22087],pam_unix(cron:session): session closed for use...
3,Nov 30 07:07:14,ip-172-31-27-153,sshd[22116],Connection closed by 122.225.103.87 [preauth]
4,Nov 30 07:07:35,ip-172-31-27-153,sshd[22118],Connection closed by 122.225.103.87 [preauth]
...,...,...,...,...
86834,Dec 31 22:26:20,ip-172-31-27-153,sshd[7998],Connection closed by 218.2.0.133 [preauth]
86835,Dec 31 22:27:07,ip-172-31-27-153,sshd[8001],Connection closed by 218.2.0.133 [preauth]
86836,Dec 31 22:27:48,ip-172-31-27-153,sshd[8003],Invalid user admin from 218.2.0.133
86837,Dec 31 22:27:48,ip-172-31-27-153,sshd[8003],input_userauth_request: invalid user admin [pr...


In [49]:
df["IP"]= df["IP"].replace("ip-172-31-27-153", "172.31.27.153")
df


,Date,IP,Service,Description
0,Nov 30 06:39:00,172.31.27.153,CRON[21882],pam_unix(cron:session): session closed for use...
1,Nov 30 06:47:01,172.31.27.153,CRON[22087],pam_unix(cron:session): session opened for use...
2,Nov 30 06:47:03,172.31.27.153,CRON[22087],pam_unix(cron:session): session closed for use...
3,Nov 30 07:07:14,172.31.27.153,sshd[22116],Connection closed by 122.225.103.87 [preauth]
4,Nov 30 07:07:35,172.31.27.153,sshd[22118],Connection closed by 122.225.103.87 [preauth]
...,...,...,...,...
86834,Dec 31 22:26:20,172.31.27.153,sshd[7998],Connection closed by 218.2.0.133 [preauth]
86835,Dec 31 22:27:07,172.31.27.153,sshd[8001],Connection closed by 218.2.0.133 [preauth]
86836,Dec 31 22:27:48,172.31.27.153,sshd[8003],Invalid user admin from 218.2.0.133
86837,Dec 31 22:27:48,172.31.27.153,sshd[8003],input_userauth_request: invalid user admin [pr...


In [50]:
df["Connecting IP"] = df["Description"].str.extract(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')
df

,Date,IP,Service,Description,Connecting IP
0,Nov 30 06:39:00,172.31.27.153,CRON[21882],pam_unix(cron:session): session closed for use...,NaN
1,Nov 30 06:47:01,172.31.27.153,CRON[22087],pam_unix(cron:session): session opened for use...,NaN
2,Nov 30 06:47:03,172.31.27.153,CRON[22087],pam_unix(cron:session): session closed for use...,NaN
3,Nov 30 07:07:14,172.31.27.153,sshd[22116],Connection closed by 122.225.103.87 [preauth],122.225.103.87
4,Nov 30 07:07:35,172.31.27.153,sshd[22118],Connection closed by 122.225.103.87 [preauth],122.225.103.87
...,...,...,...,...,...
86834,Dec 31 22:26:20,172.31.27.153,sshd[7998],Connection closed by 218.2.0.133 [preauth],218.2.0.133
86835,Dec 31 22:27:07,172.31.27.153,sshd[8001],Connection closed by 218.2.0.133 [preauth],218.2.0.133
86836,Dec 31 22:27:48,172.31.27.153,sshd[8003],Invalid user admin from 218.2.0.133,218.2.0.133
86837,Dec 31 22:27:48,172.31.27.153,sshd[8003],input_userauth_request: invalid user admin [pr...,NaN


In [59]:
df["Service"] = df["Service"].str.replace('[\[^0-9\]]', '', regex=True)
df

,Date,IP,Service,Description,Connecting IP
0,Nov 30 06:39:00,172.31.27.153,CRON,pam_unix(cron:session): session closed for use...,NaN
1,Nov 30 06:47:01,172.31.27.153,CRON,pam_unix(cron:session): session opened for use...,NaN
2,Nov 30 06:47:03,172.31.27.153,CRON,pam_unix(cron:session): session closed for use...,NaN
3,Nov 30 07:07:14,172.31.27.153,sshd,Connection closed by 122.225.103.87 [preauth],122.225.103.87
4,Nov 30 07:07:35,172.31.27.153,sshd,Connection closed by 122.225.103.87 [preauth],122.225.103.87
...,...,...,...,...,...
86834,Dec 31 22:26:20,172.31.27.153,sshd,Connection closed by 218.2.0.133 [preauth],218.2.0.133
86835,Dec 31 22:27:07,172.31.27.153,sshd,Connection closed by 218.2.0.133 [preauth],218.2.0.133
86836,Dec 31 22:27:48,172.31.27.153,sshd,Invalid user admin from 218.2.0.133,218.2.0.133
86837,Dec 31 22:27:48,172.31.27.153,sshd,input_userauth_request: invalid user admin [pr...,NaN


In [ ]:
# parse user from various lines
#def ParseUsr(line):
	#usr = None
	#if "Accepted password" in line:
		#usr = re.search(r'(\bfor\s)(\w+)', line)
	#elif "sudo:" in line:
		#usr = re.search(r'(sudo:\s+)(\w+)', line)
	#elif "authentication failure" in line:
		#usr = re.search(r'USER=\w+', line)
	#elif "for invalid user" in line:
		#usr = re.search(r'(\buser\s)(\w+)', line)
	#if usr is not None:
		#return usr.group(2)

In [ ]:
# begin parsing the passed LOG
#def ParseLogs(LOG):
	# initialize the dictionary
	#logs = {}

	# parse the log
	#f = None
	#try:
		#f = gzip.open(LOG, 'r') if '.gz' in LOG else open(LOG, 'r')
		#log = f.read()
	#except Exception, e:
		#print '[-] Error opening \'%s\': %s'%(LOG,e)
		#return None
	#finally:
		#if f is not None: f.close()

	#for line in log.split('\n'):
		# match a login
		#if "Accepted password for" in line:
			#usr = ParseUsr(line)
			
			# add 'em if they don't exist
			#if not usr in logs:
				#logs[usr] = Log(usr)
			
			#ip = ParseIP(line)
			# set info
			if not ip in logs[usr].ips:
				#logs[usr].ips.append(ip)
			#logs[usr].succ_logs.append(line.rstrip('\n'))
			#logs[usr].logs.append(line.rstrip('\n'))

		# match a failed login
		#elif "Failed password for" in line:
			# parse user
			#usr = ParseUsr(line)

			#if not usr in logs:
				#logs[usr] = Log(usr)
				
			#ip = ParseIP(line)

			#if not ip in logs[usr].ips:
				#logs[usr].ips.append(ip)
			#logs[usr].fail_logs.append(line.rstrip('\n'))
			#logs[usr].logs.append(line.rstrip('\n'))
			
		# match failed auth
		#elif ":auth): authentication failure;" in line:
			# so there are three flavors of authfail we care about;
			# su, sudo, and ssh.  Lets parse each.
			#usr = re.search(r'(\blogname=)(\w+)', line)
		#	if usr is not None:
				#usr = usr.group(2)
			# parse a fail log to ssh
			#if "(sshd:auth)" in line:
				# ssh doesn't have a logname hurr
				#usr = ParseUsr(line)
			#	if not usr in logs:
					#logs[usr] = Log(usr)
				#logs[usr].ips.append(ParseIP(line))
			# parse sudo/su fails
			#else:	
			#	if not usr in logs:
				#	logs[usr] = Log(usr)
		#	logs[usr].fail_logs.append(line.rstrip('\n'))
		#	logs[usr].logs.append(line.rstrip('\n'))
			# match commands
		#elif "sudo:" in line:
			# parse user
		#	usr = ParseUsr(line)
			#if not usr in logs:
			#	logs[usr] = Log(usr)
	
			#cmd = ParseCmd(line)
			# append the command if it isn't there already
			#if cmd is not None:
				#if not cmd in logs[usr].commands:
				#	logs[usr].commands.append(cmd)
		#	logs[usr].logs.append(line.rstrip('\n'))
	#return logs

,Month,Day
0,Nov 30 06:39:00 ip-172-31-27-153 CRON[21882]: ...,NaN
1,Nov 30 06:47:01 ip-172-31-27-153 CRON[22087]: ...,NaN
2,Nov 30 06:47:03 ip-172-31-27-153 CRON[22087]: ...,NaN
3,Nov 30 07:07:14 ip-172-31-27-153 sshd[22116]: ...,NaN
4,Nov 30 07:07:35 ip-172-31-27-153 sshd[22118]: ...,NaN
...,...,...
86834,Dec 31 22:26:20 ip-172-31-27-153 sshd[7998]: C...,NaN
86835,Dec 31 22:27:07 ip-172-31-27-153 sshd[8001]: C...,NaN
86836,Dec 31 22:27:48 ip-172-31-27-153 sshd[8003]: I...,NaN
86837,Dec 31 22:27:48 ip-172-31-27-153 sshd[8003]: i...,NaN


,Month,Day,Time,IP,Service,DescriptionWhole
0,Nov,30.0,06:39:00,ip-172-31-27-153,CRON[21882]:,NaN
1,Nov,30.0,06:47:01,ip-172-31-27-153,CRON[22087]:,pam_unix(cron:session): session opened for use...
2,Nov,30.0,06:47:03,ip-172-31-27-153,CRON[22087]:,NaN
3,Nov,30.0,07:07:14,ip-172-31-27-153,sshd[22116]:,NaN
4,Nov,30.0,07:07:35,ip-172-31-27-153,sshd[22118]:,NaN
...,...,...,...,...,...,...
86834,Dec,31.0,22:26:20,ip-172-31-27-153,sshd[7998]:,NaN
86835,Dec,31.0,22:27:07,ip-172-31-27-153,sshd[8001]:,NaN
86836,Dec,31.0,22:27:48,ip-172-31-27-153,sshd[8003]:,NaN
86837,Dec,31.0,22:27:48,ip-172-31-27-153,sshd[8003]:,NaN
